In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from itertools import cycle
from statistics import mean 
from tqdm import tqdm   

In [18]:
symbols = []
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}

In [3]:
def getProxies(inURL):
    
    page = requests.get(inURL)
    soup = BeautifulSoup(page.text, 'html.parser')
    terms = soup.find_all('tr')
    IPs = []

    for x in range(len(terms)):  
        
        term = str(terms[x])        
        
        if '<tr><td>' in str(terms[x]):
            pos1 = term.find('d>') + 2
            pos2 = term.find('</td>')

            pos3 = term.find('</td><td>') + 9
            pos4 = term.find('</td><td>US<')
            
            IP = term[pos1:pos2]
            port = term[pos3:pos4]
            
            if '.' in IP and len(port) < 6:
                IPs.append(IP + ":" + port)
                #print(IP + ":" + port)

    return IPs 


proxyURL = "https://www.us-proxy.org/"
pxs = getProxies(proxyURL)
proxyPool = cycle(pxs)

In [4]:
def clean(stng):
    
    output = ''

    for letter in stng:
        if letter != '[' and letter != ']' and letter != "'": #and letter != ' ':
            output += letter
        
    return output



def appendGrowth(lst):
    
    output = []
    
    for x in range(len(lst) - 1):
        
        try:
            diff = lst[x] - lst[x + 1]
            
            if diff > 0:
                output.append(round((lst[x] - lst[x + 1]) / abs(lst[x + 1]) * 100, 2))
            else:
                output.append(round((lst[x] - lst[x + 1]) / lst[x + 1] * 100, 2))
                
        except:
            continue
            
    return output



def getAvg(lst):
    
    summ = 0
    nums = 0
    
    for num in lst:
        if num != 0.0:
            summ += num
            nums += 1
    
    try:
        return round(summ / nums, 2)
    except:
        return 0 
    
    
def getSymbolsCSV(fileName):
    
    global symbols
    print('\nLoading data from ' + fileName)

    with open(fileName) as csvfile:
    
        readCSV = csv.reader(csvfile, delimiter=',')
        
        for row in readCSV:
            if row[0] =='ï»¿A':
                symbols.append('A')
            else:
                symbols.append(row[0])

In [20]:
fileName = 'C:\\Users\\faiza\\Automated-Fundamental-Analysis\\SymbolData\\S&P500Symbols.csv'
getSymbolsCSV(fileName)


Loading data from C:\Users\faiza\Automated-Fundamental-Analysis\SymbolData\S&P500Symbols.csv


In [22]:
overall = []

def getIncomeStatementScore(sym):
    
    url = 'https://finance.yahoo.com/quote/' + sym + '/financials?p=' + sym

    page = requests.get(url, headers=agent, proxies = {"http": next(proxyPool)})
    soup = str(BeautifulSoup(page.text, 'html.parser'))

    table = soup[soup.find('title="Total Revenue') : soup.find('People Also Watch')]
    table = table.split('data-reactid=')


    yearlyRevenues = []
    yearlyCogs = []
    yearlyNetIncome = []

    startCogs = False
    endCogs = False
    startNetIncome = False
    endNetIncome = False

    
    for line in table:

        num = line[line.find('>') + 1 : line.find('<')].replace(',','')
        numValid = 'BdB Bdc($seperatorColor)' in line
        endLine = '</div></div><div' in line

        addNum = numValid == True or endLine == True
        validCheck = len(num) != 0 and num != '-'

        if startCogs == False and addNum:

            if validCheck == True:
                yearlyRevenues.append(float(num))

            if endLine == True and len(yearlyRevenues) > 1:
                startCogs = True

        elif endCogs == False and startCogs == True and addNum:

            if validCheck == True: 
                yearlyCogs.append(float(num))

            if len(yearlyCogs) == len(yearlyRevenues):
                endCogs = True


        if 'Net Income' in line:
            startNetIncome = True

        if endNetIncome == False and startNetIncome == True and addNum:

            if validCheck == True: yearlyNetIncome.append(float(num))

            if len(yearlyNetIncome) == len(yearlyRevenues): endNetIncome = True


    print(sym + '\n')
    print('Yearly Revenues:', yearlyRevenues)
    print('Yearly Cogs:', yearlyCogs)
    print('Yearly Net Income:', yearlyNetIncome)
    print()


    grossMargins = []
    netMargins = []


    for x in range(len(yearlyRevenues)):
        grossMargins.append(round(((yearlyRevenues[x] - yearlyCogs[x]) / yearlyRevenues[x] * 100), 2))

    for x in range(len(yearlyRevenues)):
        netMargins.append(round((yearlyNetIncome[x] / yearlyRevenues[x] * 100), 2))



    yearlyRevGrowth = appendGrowth(yearlyRevenues)
    yearlyCogsGrowth = appendGrowth(yearlyCogs)
    yearlyNetIncGrowth = appendGrowth(yearlyNetIncome)
    yearlyGrossMarginGrowth = appendGrowth(grossMargins)
    yearlyNetIncMarginGrowth = appendGrowth(netMargins)

    avgYearlyRevGrowth = getAvg(yearlyRevGrowth)
    avgYearlyCogsGrowth = getAvg(yearlyCogsGrowth)
    avgNetIncGrowth = getAvg(yearlyNetIncGrowth)
    avgGrossMarginGrowth = getAvg(yearlyGrossMarginGrowth)
    avgNetIncMarginGrowth = getAvg(yearlyNetIncMarginGrowth)

    netGrowth = round(avgYearlyRevGrowth - avgYearlyCogsGrowth, 2)


    print('Yearly Rev Growth:', yearlyRevGrowth, avgYearlyRevGrowth)
    print('Yearly Cogs Growth:', yearlyCogsGrowth, avgYearlyCogsGrowth)
    print('Yearly Net Inc Growth:', yearlyNetIncGrowth, avgNetIncGrowth)

    print('\nGross Margins:', grossMargins, getAvg(grossMargins))
    print('Yearly Gross Margin Growth:', yearlyGrossMarginGrowth, avgGrossMarginGrowth)

    print('\nNet Margins:', netMargins, getAvg(netMargins))
    print('Yearly Net Margin Growth:', yearlyNetIncMarginGrowth, avgNetIncMarginGrowth)

    print('\nNet Growth', netGrowth)

    score = round(avgYearlyRevGrowth + (netGrowth * 20) + (avgGrossMarginGrowth * 5) + avgNetIncGrowth + (avgNetIncMarginGrowth * 5), 2)
    
    overall.append([sym, score, avgYearlyRevGrowth, netGrowth, avgGrossMarginGrowth, avgNetIncGrowth, avgNetIncMarginGrowth])
    
    
    print('\nFinal Score:', score)
    print()
    
    

def exportData(fileName):

    MyFile = open(fileName, 'w')

    header = 'Symbol, Score, AvgYearlyRevGrowth, NetGrowth, AvgGrossMarginGrowth, AvgNetIncGrowth, AvgNetIncMarginGrowth' + '\n'

    MyFile.write(header)

    for row in overall:
        MyFile.write(clean(str(row)))
        MyFile.write('\n')

    MyFile.close()

    print('\nSaved as ' + fileName)


In [23]:
for stock in symbols:
    getIncomeStatementScore(stock)
        
exportData('StockIncStatementRatings.csv')

AAL

Yearly Revenues: [24622000.0, 45768000.0, 44541000.0, 42207000.0]
Yearly Cogs: [27850000.0, 35379000.0, 34490000.0, 31154000.0]
Yearly Net Income: [-6293000.0, 1686000.0, 1412000.0, 1919000.0]

Yearly Rev Growth: [-46.2, 2.75, 5.53] -12.64
Yearly Cogs Growth: [-21.28, 2.58, 10.71] -2.66
Yearly Net Inc Growth: [-473.25, 19.41, -26.42] -160.09

Gross Margins: [-13.11, 22.7, 22.57, 26.19] 14.59
Yearly Gross Margin Growth: [-157.75, 0.58, -13.82] -57.0

Net Margins: [-25.56, 3.68, 3.17, 4.55] -3.54
Yearly Net Margin Growth: [-794.57, 16.09, -30.33] -269.6

Net Growth -9.98

Final Score: -2005.33

AAPL

Yearly Revenues: [273857000.0, 274515000.0, 260174000.0, 265595000.0, 229234000.0]
Yearly Cogs: [169277000.0, 169559000.0, 161782000.0, 163756000.0, 141048000.0]
Yearly Net Income: [58424000.0, 57411000.0, 55256000.0, 59531000.0, 48351000.0]

Yearly Rev Growth: [-0.24, 5.51, -2.04, 15.86] 4.77
Yearly Cogs Growth: [-0.17, 4.81, -1.21, 16.1] 4.88
Yearly Net Inc Growth: [1.76, 3.9, -7.18, 

ACN

Yearly Revenues: [44730266.0, 44327039.0, 43215013.0, 41603428.0, 36765478.0]
Yearly Cogs: [30503571.0, 30350881.0, 29900325.0, 29160515.0, 25734986.0]
Yearly Net Income: [5251147.0, 5107839.0, 4779112.0, 4059907.0, 3445149.0]

Yearly Rev Growth: [0.91, 2.57, 3.87, 13.16] 5.13
Yearly Cogs Growth: [0.5, 1.51, 2.54, 13.31] 4.46
Yearly Net Inc Growth: [2.81, 6.88, 17.71, 17.84] 11.31

Gross Margins: [31.81, 31.53, 30.81, 29.91, 30.0] 30.81
Yearly Gross Margin Growth: [0.89, 2.34, 3.01, -0.3] 1.49

Net Margins: [11.74, 11.52, 11.06, 9.76, 9.37] 10.69
Yearly Net Margin Growth: [1.91, 4.16, 13.32, 4.16] 5.89

Net Growth 0.67

Final Score: 66.74

ADI

Yearly Revenues: [5603056.0, 5603056.0, 5991065.0, 6200942.0, 5107503.0]
Yearly Cogs: [1912578.0, 1912578.0, 1977315.0, 1967640.0, 2045907.0]
Yearly Net Income: [1220761.0, 1220761.0, 1359782.0, 1489523.0, 725016.0]

Yearly Rev Growth: [0.0, -6.48, -3.38, 21.41] 3.85
Yearly Cogs Growth: [0.0, -3.27, 0.49, -3.83] -2.2
Yearly Net Inc Growth: 

ANSS

Yearly Revenues: [1445096.0, 1515892.0, 1293636.0, 1095250.0]
Yearly Cogs: [156766.0, 166273.0, 155885.0, 150164.0]
Yearly Net Income: [438606.0, 451295.0, 419375.0, 259251.0]

Yearly Rev Growth: [-4.67, 17.18, 18.11] 10.21
Yearly Cogs Growth: [-5.72, 6.66, 3.81] 1.58
Yearly Net Inc Growth: [-2.81, 7.61, 61.76] 22.19

Gross Margins: [89.15, 89.03, 87.95, 86.29] 88.11
Yearly Gross Margin Growth: [0.13, 1.23, 1.92] 1.09

Net Margins: [30.35, 29.77, 32.42, 23.67] 29.05
Yearly Net Margin Growth: [1.95, -8.17, 36.97] 10.25

Net Growth 8.63

Final Score: 261.7

AON

Yearly Revenues: [11089000.0, 11013000.0, 10770000.0, 9998000.0]
Yearly Cogs: [5992000.0, 6054000.0, 6103000.0, 6089000.0]
Yearly Net Income: [1645000.0, 1532000.0, 1134000.0, 1226000.0]

Yearly Rev Growth: [0.69, 2.26, 7.72] 3.56
Yearly Cogs Growth: [-1.02, -0.8, 0.23] -0.53
Yearly Net Inc Growth: [7.38, 35.1, -7.5] 11.66

Gross Margins: [45.96, 45.03, 43.33, 39.1] 43.35
Yearly Gross Margin Growth: [2.07, 3.92, 10.82] 5.6


ATVI

Yearly Revenues: [6452000.0, 6489000.0, 7500000.0, 7017000.0]
Yearly Cogs: [2036000.0, 2094000.0, 2517000.0, 2501000.0]
Yearly Net Income: [1561000.0, 1503000.0, 1813000.0, 273000.0]

Yearly Rev Growth: [-0.57, -13.48, 6.88] -2.39
Yearly Cogs Growth: [-2.77, -16.81, 0.64] -6.31
Yearly Net Inc Growth: [3.86, -17.1, 564.1] 183.62

Gross Margins: [68.44, 67.73, 66.44, 64.36] 66.74
Yearly Gross Margin Growth: [1.05, 1.94, 3.23] 2.07

Net Margins: [24.19, 23.16, 24.17, 3.89] 18.85
Yearly Net Margin Growth: [4.45, -4.18, 521.34] 173.87

Net Growth 3.92

Final Score: 1139.33

AVGO

Yearly Revenues: [22891000.0, 23888000.0, 22597000.0, 20848000.0, 17636000.0]
Yearly Cogs: [10252000.0, 10372000.0, 10114000.0, 10115000.0, 9127000.0]
Yearly Net Income: [2332000.0, 2663000.0, 2695000.0, 12259000.0, 1692000.0]

Yearly Rev Growth: [-4.17, 5.71, 8.39, 18.21] 7.04
Yearly Cogs Growth: [-1.16, 2.55, -0.01, 10.83] 3.05
Yearly Net Inc Growth: [-12.43, -1.19, -78.02, 624.53] 133.22

Gross Margins: [5

BA

Yearly Revenues: [60765000.0, 76559000.0, 101127000.0, 93392000.0]
Yearly Cogs: [61559000.0, 72093000.0, 81490000.0, 76066000.0]
Yearly Net Income: [-4463000.0, -636000.0, 10453000.0, 8191000.0]

Yearly Rev Growth: [-20.63, -24.29, 8.28] -12.21
Yearly Cogs Growth: [-14.61, -11.53, 7.13] -6.34
Yearly Net Inc Growth: [601.73, -106.08, 27.62] 174.42

Gross Margins: [-1.31, 5.83, 19.42, 18.55] 10.62
Yearly Gross Margin Growth: [-122.47, -69.98, 4.69] -62.59

Net Margins: [-7.34, -0.83, 10.34, 8.77] 2.73
Yearly Net Margin Growth: [784.34, -108.03, 17.9] 231.4

Net Growth -5.87

Final Score: 888.86

BAX

Yearly Revenues: [11531000.0, 11362000.0, 11127000.0, 10561000.0]
Yearly Cogs: [6837000.0, 6601000.0, 6346000.0, 6099000.0]
Yearly Net Income: [911000.0, 1001000.0, 1624000.0, 717000.0]

Yearly Rev Growth: [1.49, 2.11, 5.36] 2.99
Yearly Cogs Growth: [3.58, 4.02, 4.05] 3.88
Yearly Net Inc Growth: [-8.99, -38.36, 126.5] 26.38

Gross Margins: [40.71, 41.9, 42.97, 42.25] 41.96
Yearly Gross M

...

DVA

Yearly Revenues: [11543866.0, 11388479.0, 11404851.0, 10876634.0]
Yearly Cogs: [7932357.0, 7914485.0, 8195513.0, 7640005.0]
Yearly Net Income: [844740.0, 810981.0, 159394.0, 663618.0]

Yearly Rev Growth: [1.36, -0.14, 4.86] 2.03
Yearly Cogs Growth: [0.23, -3.43, 7.27] 1.36
Yearly Net Inc Growth: [4.16, 408.79, -75.98] 112.32

Gross Margins: [31.29, 30.5, 28.14, 29.76] 29.92
Yearly Gross Margin Growth: [2.59, 8.39, -5.44] 1.85

Net Margins: [7.32, 7.12, 1.4, 6.1] 5.49
Yearly Net Margin Growth: [2.81, 408.57, -77.05] 111.44

Net Growth 0.67

Final Score: 694.2

DXC

Yearly Revenues: [18892000.0, 19577000.0, 20753000.0, 24556000.0, 7607000.0]
Yearly Cogs: [14792000.0, 14901000.0, 14946000.0, 17944000.0, 5545000.0]
Yearly Net Income: [-3862000.0, -5369000.0, 1257000.0, 1751000.0, -123000.0]

Yearly Rev Growth: [-3.5, -5.67, -15.49, 222.81] 49.54
Yearly Cogs Growth: [-0.73, -0.3, -16.71, 223.61] 51.47
Yearly Net Inc Growth: [28.07, -527.13, -28.21, 1523.58] 249.08

Gross Margins: [21.7

ED

Yearly Revenues: [12237000.0, 12574000.0, 12337000.0, 12033000.0]
Yearly Cogs: [5145000.0, 5808000.0, 6100000.0, 5928000.0]
Yearly Net Income: [1353000.0, 1343000.0, 1382000.0, 1525000.0]

Yearly Rev Growth: [-2.68, 1.92, 2.53] 0.59
Yearly Cogs Growth: [-11.42, -4.79, 2.9] -4.44
Yearly Net Inc Growth: [0.74, -2.82, -9.38] -3.82

Gross Margins: [57.96, 53.81, 50.56, 50.74] 53.27
Yearly Gross Margin Growth: [7.71, 6.43, -0.35] 4.6

Net Margins: [11.06, 10.68, 11.2, 12.67] 11.4
Yearly Net Margin Growth: [3.56, -4.64, -11.6] -4.23

Net Growth 5.03

Final Score: 99.22

EIX

Yearly Revenues: [13391000.0, 12347000.0, 12657000.0, 12320000.0]
Yearly Cogs: [8456000.0, 7857000.0, 8203000.0, 7680000.0]
Yearly Net Income: [356000.0, 1284000.0, -423000.0, 565000.0]

Yearly Rev Growth: [8.46, -2.45, 2.74] 2.92
Yearly Cogs Growth: [7.62, -4.22, 6.81] 3.4
Yearly Net Inc Growth: [-72.27, 403.55, -174.87] 52.14

Gross Margins: [36.85, 36.37, 35.19, 37.66] 36.52
Yearly Gross Margin Growth: [1.32, 3.35

ESS

Yearly Revenues: [1509040.0, 1460155.0, 1400053.0, 1363899.0]
Yearly Cogs: [465438.0, 430594.0, 416396.0, 405542.0]
Yearly Net Income: [601943.0, 439286.0, 390153.0, 433059.0]

Yearly Rev Growth: [3.35, 4.29, 2.65] 3.43
Yearly Cogs Growth: [8.09, 3.41, 2.68] 4.73
Yearly Net Inc Growth: [37.03, 12.59, -9.91] 13.24

Gross Margins: [69.16, 70.51, 70.26, 70.27] 70.05
Yearly Gross Margin Growth: [-1.91, 0.36, -0.01] -0.52

Net Margins: [39.89, 30.08, 27.87, 31.75] 32.4
Yearly Net Margin Growth: [32.61, 7.93, -12.22] 9.44

Net Growth -1.3

Final Score: 35.27

ETR

Yearly Revenues: [10205840.0, 10878673.0, 11009452.0, 11074481.0]
Yearly Cogs: [5754195.0, 6699806.0, 7839136.0, 7549751.0]
Yearly Net Income: [1385391.0, 1241226.0, 848661.0, 411612.0]

Yearly Rev Growth: [-6.18, -1.19, -0.59] -2.65
Yearly Cogs Growth: [-14.11, -14.53, 3.83] -8.27
Yearly Net Inc Growth: [11.61, 46.26, 106.18] 54.68

Gross Margins: [43.62, 38.41, 28.8, 31.83] 35.66
Yearly Gross Margin Growth: [13.56, 33.37, -9

FTNT

Yearly Revenues: [2354300.0, 2156200.0, 1801200.0, 1494930.0]
Yearly Cogs: [528400.0, 505900.0, 450400.0, 385284.0]
Yearly Net Income: [411100.0, 326500.0, 332200.0, 31399.0]

Yearly Rev Growth: [9.19, 19.71, 20.49] 16.46
Yearly Cogs Growth: [4.45, 12.32, 16.9] 11.22
Yearly Net Inc Growth: [25.91, -1.72, 958.0] 327.4

Gross Margins: [77.56, 76.54, 74.99, 74.23] 75.83
Yearly Gross Margin Growth: [1.33, 2.07, 1.02] 1.47

Net Margins: [17.46, 15.14, 18.44, 2.1] 13.29
Yearly Net Margin Growth: [15.32, -17.9, 778.1] 258.51

Net Growth 5.24

Final Score: 1748.56

GD

Yearly Revenues: [38217000.0, 39350000.0, 36193000.0, 30973000.0]
Yearly Cogs: [31785000.0, 32291000.0, 29478000.0, 24786000.0]
Yearly Net Income: [3185000.0, 3484000.0, 3345000.0, 2912000.0]

Yearly Rev Growth: [-2.88, 8.72, 16.85] 7.56
Yearly Cogs Growth: [-1.57, 9.54, 18.93] 8.97
Yearly Net Inc Growth: [-8.58, 4.16, 14.87] 3.48

Gross Margins: [16.83, 17.94, 18.55, 19.98] 18.32
Yearly Gross Margin Growth: [-6.19, -3.29,

KeyboardInterrupt: 